# Cleaning the data and providing a small and easily parsable database.

The database with the full logs is too huge to work on. Most of the data would only be needed for debugging but not for evaluation.
Thus, we create a clean and small database for the further evaluation.

In [1]:
import pandas as pd
from _conf import EXTENDED_BASELINE_DATA, RESULT_FOLDER
from algbench import read_as_pandas

In [2]:
# Read the baseline results for comparison. They also already contain instance data (num_clauses and num_features).
baseline_data = pd.read_json(EXTENDED_BASELINE_DATA)
baseline_data.rename(columns={"sample_size": "baseline_sample_size"}, inplace=True)
baseline_data

,instance_name,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime,baseline_sample_size,path_to_baseline_sample,archive_path
0,PPU,52.0,109.0,FIDE-ICPL,t2,1706,15.0,2024-02-23_21-52-24/1_1_0_1_sample.csv,./EXTERNAL_INPUT/baseline.zip
1,PPU,52.0,109.0,FIDE-Chvatal,t2,1455,15.0,2024-02-23_21-52-24/1_1_1_1_sample.csv,./EXTERNAL_INPUT/baseline.zip
2,PPU,52.0,109.0,Incling,t2,171,17.0,2024-02-23_21-52-24/1_1_2_1_sample.csv,./EXTERNAL_INPUT/baseline.zip
3,PPU,52.0,109.0,FIDE-YASA,t2_m1_null,266,13.0,2024-02-23_21-52-24/1_1_3_1_sample.csv,./EXTERNAL_INPUT/baseline.zip
4,PPU,52.0,109.0,FIDE-YASA,t2_m3_null,290,12.0,2024-02-23_21-52-24/1_1_4_1_sample.csv,./EXTERNAL_INPUT/baseline.zip
...,...,...,...,...,...,...,...,...,...
2825,am31_sim,1178.0,2747.0,FIDE-YASA(15min),,900000,69.0,/home/krupke/Repositories/SampLNS/evaluation/0...,None
2826,am31_sim,1178.0,2747.0,FIDE-YASA(15min),,900000,72.0,/home/krupke/Repositories/SampLNS/evaluation/0...,None
2827,am31_sim,1178.0,2747.0,FIDE-YASA(15min),,900000,70.0,/home/krupke/Repositories/SampLNS/evaluation/0...,None
2828,am31_sim,1178.0,2747.0,FIDE-YASA(15min),,900000,72.0,/home/krupke/Repositories/SampLNS/evaluation/0...,None


In [3]:
# Get the data from the result folder
samplns_data = read_as_pandas(
    RESULT_FOLDER,
    lambda result: {
        "parameters": result["parameters"],
        "initial_sample_path": result["parameters"]["args"]["initial_sample_path"],
        "instance_name": result["parameters"]["args"]["instance_name"],
        "lower_bound": result["result"]["lower_bound"],
        "upper_bound": len(result["result"]["solution"]),
        "iteration_info": result["result"]["iteration_info"],
        "time_used_by_yasa": result["result"]["time_used_by_yasa"],
        "timelimit_for_samplns": result["result"]["timelimit_for_samplns"],
        "samplns_used": result["result"]["samplns_used"],
        "runtime": result["runtime"],
        "time_limit": result["parameters"]["args"]["time_limit"],
    }
    if result.get("result", None) and result.get("result", {}).get("solution", None)
    else None,
)
samplns_data.drop_duplicates(subset=["initial_sample_path"], inplace=True)
samplns_data

,parameters,initial_sample_path,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime,time_limit
0,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/4_1_3_1_sample.csv,ChatClient,7.0,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",0.205,10799.795,True,8.636926,10800
1,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/16_1_3_5_sample.csv,PPU,12.0,12,"[{'nbrhd_tuples': 164, 'nbrhd_confs': 7, 'iter...",0.289,10799.711,True,6.240633,10800
2,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/6_1_3_4_sample.csv,car,5.0,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 6, 'itera...",0.155,10799.845,True,0.062622,10800
3,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/6_1_3_2_sample.csv,car,5.0,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 6, 'itera...",0.174,10799.826,True,0.043420,10800
4,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/47_1_3_3_sample.csv,ea2468,31.0,39,"[{'nbrhd_tuples': 178, 'nbrhd_confs': 6, 'iter...",4.993,10795.007,True,10938.016142,10800
...,...,...,...,...,...,...,...,...,...,...,...
225,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/33_1_3_3_sample.csv,fs_2017-05-22,396.0,396,"[{'nbrhd_tuples': 0, 'nbrhd_confs': 0, 'iterat...",2.070,10797.930,True,433.986984,10800
226,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/37_1_3_1_sample.csv,busybox-1_29_2,18.0,22,"[{'nbrhd_tuples': 228, 'nbrhd_confs': 4, 'iter...",4.184,10795.816,True,10859.846727,10800
227,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/17_1_3_4_sample.csv,berkeleyDB1,15.0,15,"[{'nbrhd_tuples': 204, 'nbrhd_confs': 10, 'ite...",0.337,10799.663,True,559.524486,10800
228,"{'func': 'run_samplns', 'args': {'instance_nam...",2023-03-01_13-51-03/9_1_3_5_sample.csv,APL,7.0,7,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 10, 'ite...",0.300,10799.700,True,8.174069,10800


In [4]:
# Pull the baseline info into the data frame
samplns_data_with_baseline_infos = baseline_data.merge(
    right=samplns_data,
    left_on="path_to_baseline_sample",
    right_on="initial_sample_path",
    how="right",
    suffixes=("_baseline", "_samplns"),
)
samplns_data_with_baseline_infos.drop(
    columns=["instance_name_baseline", "initial_sample_path"], inplace=True
)
samplns_data_with_baseline_infos.rename(
    columns={"instance_name_samplns": "instance_name"}, inplace=True
)
samplns_data_with_baseline_infos

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,archive_path,parameters,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime_samplns,time_limit
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",ChatClient,7.0,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",0.205,10799.795,True,8.636926,10800
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",PPU,12.0,12,"[{'nbrhd_tuples': 164, 'nbrhd_confs': 7, 'iter...",0.289,10799.711,True,6.240633,10800
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",car,5.0,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 6, 'itera...",0.155,10799.845,True,0.062622,10800
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",car,5.0,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 6, 'itera...",0.174,10799.826,True,0.043420,10800
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,31.0,39,"[{'nbrhd_tuples': 178, 'nbrhd_confs': 6, 'iter...",4.993,10795.007,True,10938.016142,10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",fs_2017-05-22,396.0,396,"[{'nbrhd_tuples': 0, 'nbrhd_confs': 0, 'iterat...",2.070,10797.930,True,433.986984,10800
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",busybox-1_29_2,18.0,22,"[{'nbrhd_tuples': 228, 'nbrhd_confs': 4, 'iter...",4.184,10795.816,True,10859.846727,10800
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",berkeleyDB1,15.0,15,"[{'nbrhd_tuples': 204, 'nbrhd_confs': 10, 'ite...",0.337,10799.663,True,559.524486,10800
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",APL,7.0,7,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 10, 'ite...",0.300,10799.700,True,8.174069,10800


In [5]:
# Check that we have all results.
samplns_data_with_baseline_infos.groupby(["instance_name"]).size()

instance_name
APL                            5
APL-Model                      5
BankingSoftware                5
BattleofTanks                  5
ChatClient                     5
DMIE                           5
E-Shop                         5
EMBToolkit                     5
FameDB                         5
FeatureIDE                     5
FreeBSD-8_0_0                  5
PPU                            5
SafeBali                       5
SortingLine                    5
TightVNC                       5
Violet                         5
WaterlooGenerated              5
XSEngine                       5
aaed2000                       5
am31_sim                       5
atlas_mips32_4kc               5
axTLS                          5
berkeleyDB1                    5
berkeleyDB2                    5
busybox-1_18_0                 5
busybox-1_29_2                 5
busybox_2007-01-24_09-14-09    5
busybox_2020-12-16_21-53-05    5
calculate                      5
car                          

In [6]:
# Will stop the notebook if the data is bad
assert samplns_data_with_baseline_infos.groupby(["instance_name"]).size().min() == 5
assert samplns_data_with_baseline_infos.groupby(["instance_name"]).size().max() == 5

In [7]:
# Save the cleaned data to a json file
from _conf import PREPROCESSDED_RESULTS

samplns_data_with_baseline_infos.to_json(PREPROCESSDED_RESULTS)

In [8]:
# This just for looking into one instance.
t = read_as_pandas(
    RESULT_FOLDER,
    lambda result: {
        "parameters": result["parameters"],
        "initial_sample_path": result["parameters"]["args"]["initial_sample_path"],
        "instance_name": result["parameters"]["args"]["instance_name"],
        "lower_bound": result["result"]["lower_bound"],
        "upper_bound": len(result["result"]["solution"]),
        "iteration_info": result["result"]["iteration_info"],
        "time_used_by_yasa": result["result"]["time_used_by_yasa"],
        "timelimit_for_samplns": result["result"]["timelimit_for_samplns"],
        "samplns_used": result["result"]["samplns_used"],
        "instance": result["parameters"]["args"]["instance_name"],
        "logging": result["logging"],
    }
    if result.get("result", None) and result.get("result", {}).get("solution", None)
    else None,
)
t = t[t["instance_name"] == "toybox_2020-12-06_00-02-46"]
for e in t.iloc[0]["logging"]:
    print(e["message"])

Parsed instance 'models/toybox_2020-12-06_00-02-46' with 334 features and 92 rules.
Preprocessing instance (Instance[models/toybox_2020-12-06_00-02-46]<334 features, 92 rules>).
Converting instance to CNF (Instance[models/toybox_2020-12-06_00-02-46|EQ]<334 features, 92 rules>).
Finished converting instance to CNF (Instance[models/toybox_2020-12-06_00-02-46|EQ|CNF]<334 features, 92 rules>).
Finnished preprocessing (Instance[models/toybox_2020-12-06_00-02-46|EQ|CNF]<334 features, 92 rules>).
Setting up random neighborhood selector.
Computing feasible tuples...
Converted sample to list representation.
Instance has 206665 feasible tuples.
Neighborhood selector is ready.
Building transaction graph for models/toybox_2020-12-06_00-02-46|EQ|CNF with 334 concrete features!
All valid configurations were added to the transaction graph.
Setting up random neighborhood selector.
Computing feasible tuples...
Converted sample to list representation.
Instance has 206665 feasible tuples.
Neighborhood se

In [9]:
# Check that we can read the data
data = pd.read_json(PREPROCESSDED_RESULTS)
data

,num_vars,num_clauses,baseline_alg,baseline_alg_conf,runtime_baseline,baseline_sample_size,path_to_baseline_sample,archive_path,parameters,instance_name,lower_bound,upper_bound,iteration_info,time_used_by_yasa,timelimit_for_samplns,samplns_used,runtime_samplns,time_limit
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",ChatClient,7,7,"[{'nbrhd_tuples': 176, 'nbrhd_confs': 10, 'ite...",0.205,10799.795,True,8.636926,10800
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",PPU,12,12,"[{'nbrhd_tuples': 164, 'nbrhd_confs': 7, 'iter...",0.289,10799.711,True,6.240633,10800
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",car,5,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 6, 'itera...",0.155,10799.845,True,0.062622,10800
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",car,5,5,"[{'nbrhd_tuples': 77, 'nbrhd_confs': 6, 'itera...",0.174,10799.826,True,0.043420,10800
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",ea2468,31,39,"[{'nbrhd_tuples': 178, 'nbrhd_confs': 6, 'iter...",4.993,10795.007,True,10938.016142,10800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",fs_2017-05-22,396,396,"[{'nbrhd_tuples': 0, 'nbrhd_confs': 0, 'iterat...",2.070,10797.930,True,433.986984,10800
226,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",busybox-1_29_2,18,22,"[{'nbrhd_tuples': 228, 'nbrhd_confs': 4, 'iter...",4.184,10795.816,True,10859.846727,10800
227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",berkeleyDB1,15,15,"[{'nbrhd_tuples': 204, 'nbrhd_confs': 10, 'ite...",0.337,10799.663,True,559.524486,10800
228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'func': 'run_samplns', 'args': {'instance_nam...",APL,7,7,"[{'nbrhd_tuples': 207, 'nbrhd_confs': 10, 'ite...",0.300,10799.700,True,8.174069,10800


* `instance_name`: The instance name.
* `num_vars`: The number of variables in the instance.
* `num_clauses`: The number of clauses in the instance.
* `baseline_alg`: The baseline algorithm that created the sample
* `baseline_alg_conf`: The configuration of the baseline algorithm that created the sample.
* `runtime_baseline`: The runtime of the baseline algorithm that created the sample.
* `baseline_sample_size`: The size of the sample created by the baseline algorithm.
* `path_to_baseline_sample`: The path to the sample created by the baseline algorithm.
* `parameters` The parameters used to run samplns.
* `lower_bound` The lower bound computed by SampLNS.
* `upper_bound` The upper bound computed by SampLNS.
* `iteration_info` Info the extract the progress of SampLNS. The time do not include the baseline algorithm.
* `time_used_by_yasa` The time used by the baseline algorithm.
* `timelimit_for_samplns` The time limit for SampLNS. Check also `parameters` for the full timelimit.
* `runtime_samplns` The runtime of SampLNS without the baseline.
* `time_limit` The overall timelimit.
